# Fish Speech

### For Linux User / Linux 用户

In [1]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

### Prepare Model

In [3]:
# For Chinese users, you probably want to use mirror to accelerate downloading
# !set HF_ENDPOINT=https://hf-mirror.com
# !export HF_ENDPOINT=https://hf-mirror.com 

!huggingface-cli download fishaudio/fish-speech-1.5 --local-dir checkpoints/fish-speech-1.5/

Fetching 7 files:   0%|                                   | 0/7 [00:00<?, ?it/s]Downloading 'model.pth' to 'checkpoints/fish-speech-1.5/.cache/huggingface/download/YT0Y2lJH9mHYafdr2d9j82hXvzY=.918dc960372cc1b77bbafb14c48ef7a1634ecf75d4eb85b78607223b780d6001.incomplete'

.gitattributes:   0%|                               | 0.00/1.52k [00:00<?, ?B/s]
.gitattributes: 100%|██████████████████████| 1.52k/1.52k [00:00<00:00, 2.72MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.5/.gitattributes
special_tokens.json: 100%|█████████████████| 31.0k/31.0k [00:00<00:00, 54.9MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.5/special_tokens.json
config.json: 100%|█████████████████████████████| 697/697 [00:00<00:00, 5.62MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.5/config.json

README.md: 100%|███████████████████████████| 1.68k/1.68k [00:00<00:00, 9.08MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.5/README.md

(…)fly-gan-vq-fsq

## WebUI Inference

> You can use --compile to fuse CUDA kernels for faster inference (10x).

In [ ]:
!python tools/run_webui.py \
    --llama-checkpoint-path checkpoints/fish-speech-1.5-guj-lora \
    --decoder-checkpoint-path checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth \
    --compile

2025-06-09 07:57:51.803 | INFO     | __main__:<module>:56 - Loading Llama model...
2025-06-09 07:58:06.789 | INFO     | fish_speech.models.text2semantic.inference:load_model:681 - Restored model from checkpoint
2025-06-09 07:58:06.789 | INFO     | fish_speech.models.text2semantic.inference:load_model:687 - Using DualARTransformer
2025-06-09 07:58:06.789 | INFO     | fish_speech.models.text2semantic.inference:load_model:695 - Compiling function...
2025-06-09 07:58:08.130 | INFO     | __main__:<module>:64 - Loading VQ-GAN model...
/home/bhavsar_nisarg_nobroker_in/miniconda3/envs/fish-speech/lib/python3.11/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:445: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/bhavsar_nisarg_nobroker_in/miniconda3/envs/fish-speech/lib/python3.11/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:630: FutureWarning: `torch

## Break-down CLI Inference

### 1. Encode reference audio: / 从语音生成 prompt: 

You should get a `fake.npy` file.

你应该能得到一个 `fake.npy` 文件.

In [ ]:
## Enter the path to the audio file here
src_audio = r"D:\PythonProject\vo_hutao_draw_appear.wav"

!python fish_speech/models/vqgan/inference.py \
    -i {src_audio} \
    --checkpoint-path "checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

### 2. Generate semantic tokens from text: / 从文本生成语义 token:

> This command will create a codes_N file in the working directory, where N is an integer starting from 0.

> You may want to use `--compile` to fuse CUDA kernels for faster inference (~30 tokens/second -> ~300 tokens/second).

> 该命令会在工作目录下创建 codes_N 文件, 其中 N 是从 0 开始的整数.

> 您可以使用 `--compile` 来融合 cuda 内核以实现更快的推理 (~30 tokens/秒 -> ~300 tokens/秒)

In [ ]:
!python fish_speech/models/text2semantic/inference.py \
    --text "hello world" \
    --prompt-text "The text corresponding to reference audio" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.5" \
    --num-samples 2
    # --compile

### 3. Generate speech from semantic tokens: / 从语义 token 生成人声:

In [ ]:
!python fish_speech/models/vqgan/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)